# <center>casased - Casablanca Stock Exchange Data Retriever</center>

This notebook demonstrates the features of the `casased` Python library for retrieving historical and intraday data from the Casablanca Stock Exchange (Bourse de Casablanca).

**Data Source:** [Medias24 API](https://medias24.com/bourse)

## Features
- 📈 Historical price data for stocks and indices (MASI, MSI20)
- ⏱️ Intraday price data
- 🏦 Multiple asset loading
- 📋 Session data, key indicators, dividends
- 📉 Index summaries and weights

In [12]:
# Install casased (uncomment if needed)
# !pip install casased

import importlib
import casased as cas
import casased.load
import pandas as pd

# Reload to pick up any local changes
importlib.reload(casased.load)
importlib.reload(cas)

print("casased imported successfully")

casased imported successfully


## 1. Available Assets

Get the list of all available stock names and their ISIN codes.

In [2]:
# List all available assets
assets = cas.notation()
print(f"Total assets available: {len(assets)}")
print(assets[:10])  # Show first 10

Total assets available: 74
['Addoha', 'AFMA', 'Afric Indus', 'Afriquia Gaz', 'Agma', 'Akdital', 'Alliances', 'Aluminium Maroc', 'Aradei Capital', 'ATLANTASANAD']


In [3]:
# Get ISIN code for a specific asset
isin = cas.get_isin_by_name('BCP')
print(f"ISIN for BCP: {isin}")

ISIN for BCP: MA0000011884


## 2. Historical Price Data

Retrieve historical OHLCV data for any listed stock using the Medias24 API.

In [7]:
# Load historical data for a single stock
data = cas.get_history('BCP', start='2024-01-01', end='2024-12-31')
print(f"Data shape: {data.shape}")
print(f"Columns: {list(data.columns)}")
data.head(10)

Data shape: (246, 5)
Columns: ['Value', 'Min', 'Max', 'Variation', 'Volume']


,Value,Min,Max,Variation,Volume
Date,,,,,
2024-01-02,276.95,275.00,276.95,0.71,7693
2024-01-03,275.00,274.50,276.95,-0.70,44584
2024-01-04,276.00,275.00,276.00,0.36,25820
2024-01-05,279.95,275.05,279.95,1.43,29925
2024-01-08,280.00,277.00,280.00,0.02,30851
2024-01-09,282.95,278.00,283.00,1.05,57578
2024-01-10,284.50,282.00,288.00,0.55,103671
2024-01-12,285.00,282.15,285.00,0.18,67089
2024-01-15,284.00,284.00,285.00,-0.35,44784


In [8]:
# Load MASI index history
masi = cas.get_history('MASI')
print(f"MASI data shape: {masi.shape}")
masi.tail()

MASI data shape: (1247, 1)


,Value
2025-12-29,18386.4872
2025-12-30,18654.7425
2025-12-31,18846.3502
2026-01-05,19440.9132
2026-01-06,19554.9716


In [9]:
# Load MSI20 index history
msi20 = cas.get_history('MSI20')
print(f"MSI20 data shape: {msi20.shape}")
msi20.tail()

MSI20 data shape: (1247, 1)


,Value
2025-12-29,1451.809149
2025-12-30,1474.866343
2025-12-31,1485.647215
2026-01-05,1537.715828
2026-01-06,1543.208520


## 3. Loading Multiple Assets

Load data for multiple stocks at once and compare.

In [10]:
# Load multiple assets - closing prices
# Note: Some assets may fail due to API rate limiting
banks = cas.loadmany(['BCP', 'Attijariwafa', 'BMCI'], 
                      start='2024-01-01', end='2024-06-30')
print(f"Shape: {banks.shape}")
banks.dropna().head()

Shape: (246, 5)


,BCP,BMCI,BOA,CIH,Attijariwafa
Date,,,,,
2024-01-02,276.95,548.8,NaN,NaN,462.5
2024-01-03,275.00,557.9,NaN,NaN,465.2
2024-01-04,276.00,560.0,NaN,NaN,467.6
2024-01-05,279.95,570.0,NaN,NaN,470.0
2024-01-08,280.00,570.0,NaN,NaN,470.0


In [ ]:
# Load volumes for multiple assets
volumes = cas.loadmany(['BCP', 'Attijariwafa'], 
                        start='2024-01-01', end='2024-03-31', 
                        feature='Volume')
volumes.dropna().head()

,BCP,BMCI,BOA,CIH


## 4. Intraday Data

Get real-time intraday price data for stocks and indices.

In [15]:
# Get intraday data for a stock
# Note: Intraday data availability depends on market hours and API accessibility
intraday = cas.get_intraday('BCP')
if not intraday.empty:
    print(f"Intraday data shape: {intraday.shape}")
    display(intraday.head(10))
else:
    print("Intraday data not available (market may be closed or API temporarily blocked)")

Status: 403
Content type: text/html; charset=UTF-8
Text sample: <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helve


In [14]:
# Get MASI intraday
masi_intraday = cas.get_intraday('MASI')
if not masi_intraday.empty and 'Value' in masi_intraday.columns:
    print(f"MASI intraday shape: {masi_intraday.shape}")
    display(masi_intraday.head())
else:
    print("MASI intraday data not available")

MASI intraday shape: (1, 2)


,result,message
0,"{'labels': ['09:30', '09:31', '09:32', '09:33'...",200 OK


## 5. Session Data & Key Indicators

Get detailed session data, key indicators, and dividend history (from Bourse de Casablanca).

In [ ]:
# Get session data for a stock
cours = cas.getCours('BOA')
print("Available keys:", list(cours.keys()))
cours.get('Données_Seance', {})

{}

In [ ]:
# Get key indicators
indicators = cas.getKeyIndicators('Attijariwafa')
print("Available keys:", list(indicators.keys()))
indicators.get('Info_Societe', {})

Could not fetch index recap: HTTPSConnectionPool(host='www.casablanca-bourse.com', port=443): Max retries exceeded with url: /bourseweb/index.aspx (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1033)')))


dict_keys([])

In [ ]:
# Get dividend history
dividends = cas.getDividend('Attijariwafa')
pd.DataFrame(dividends)

## 6. Index Information

Get market index summaries, weights, and session recaps.

In [ ]:
# Get all index summaries
index_data = cas.getIndex()
print("Available keys:", list(index_data.keys()))
index_data.get('Resume indice', {}).get('MASI', {})

In [ ]:
# Get index weights (pondération)
weights = cas.getPond()
pd.DataFrame(weights).head(10)

In [ ]:
# Get session recap
recap = cas.getIndexRecap()
print("Available keys:", list(recap.keys()) if isinstance(recap, dict) else type(recap))
recap